# **PRÁCTICA 4:** Resolución aproximada de ecuaciones mediante los métodos de Horner y Sturm

## 1. Método de Horner

Implementar el Método de Horner para que, dado un polinomio $P(x)$ de
cualquier grado y un punto $x_0 \in \mathbb{R}$, se determine otro polinomio $Q(x)$ tal que

$$ P(x) = (x-x_0) \: Q(x) + P(x_0) $$

In [1]:
import numpy as np

In [2]:
def Horner(P,x_0):
    res = np.zeros([len(P)])
    res[0] = P[0]
    for i in range(1, len(P)):
        res[i] = (res[i - 1] * x_0) + P[i]
    Q = res[0:(len(P) - 1)]
    resto = res[-1]
    return Q, resto

Utilizarla en:

$$P(x) = 5x^3+7x^2+2x+3$$ 
con 
$x_0=0$ y $x_0 = 3$

In [3]:
P = np.array([5, 7, 2, 3])
Q, resto = Horner(P, 3)
print(f'Polinomio: {Q}, resto: {resto}')

Polinomio: [ 5. 22. 68.], resto: 207.0


## 2. Método de Sturm

1. Implementar el Método de Sturm para, dado un polinomio $P(x)$, **construir la secuencia de polinomios de Sturm**. Hacer una función que devuelva una lista de polinomios (ndarrays).

In [4]:
def SturmSequence(P):
    S = list()
    S.append(P) # Primer polinomio de sturm, el propio polinomio
    S.append(np.polyder(P)) # Segundo polinomio de sturm, la derivada del polinomio
    _, resto = np.polydiv(S[0], S[1]) # Tercer polinomio de sturm
    S.append(resto)
    cont = 2
    while len(resto) > 1:
        _, resto = np.polydiv(S[cont - 1], S[cont]) # Sucesivos polinomios de la secuencia 
                                                    # hasta tener unicamente un termino independiente
        S.append(resto)
        cont += 1
    return S

In [5]:
seq = SturmSequence(P)
seq

[array([5, 7, 2, 3]),
 array([15, 14,  2]),
 array([-0.84444444,  2.68888889]),
 array([198.66689751])]

2. Complementar la implementación del apartado 1, para, utilizando el Método de Sturm, **hallar el número de raíces positivas y negativas** de $P(x)$ en un intervalo dado. Hacer una función que devuelva los valores `pos` y `neg`.

NOTA: Asumir que todas las raíces son simples. 

In [23]:
def numberOfRoots(P, interval):
    # Initial values
    sequence = SturmSequence(P)
    pos = 0
    neg = 0
    neg_changes = 0
    mid_changes = 0
    pos_changes = 0
    iter_neg_val = 0
    iter_mid_val = 0
    iter_pos_val = 0

    # Previous iteration values
    prev_neg_val = np.polyval(sequence[0], interval[0])
    prev_mid_val = np.polyval(sequence[0], interval[1])
    prev_pos_val = np.polyval(sequence[0], interval[2])

    # Start algorithm
    for elem in sequence:
        iter_neg_val = np.polyval(elem, interval[0])
        iter_mid_val = np.polyval(elem, interval[1])
        iter_pos_val = np.polyval(elem, interval[2])

        if (prev_neg_val * iter_neg_val < 0):
            neg_changes += 1
        if (prev_mid_val * iter_mid_val < 0):
            mid_changes += 1
        if (prev_pos_val * iter_pos_val < 0):
            pos_changes += 1
        
        prev_neg_val = iter_neg_val
        prev_mid_val = iter_mid_val
        prev_pos_val = iter_pos_val

    neg = neg_changes - mid_changes
    pos = mid_changes - pos_changes

    print(f'Cambios de signo en {interval[0]} = [{neg_changes}], {interval[1]} = [{mid_changes}] y {interval[2]} = [{pos_changes}]')
    print(f'Raices negativas = [raices en {interval[0]}] {neg_changes} - [raices en {interval[1]}] {mid_changes} = {neg}')
    print(f'Raices positivas = [raices en {interval[1]}] {mid_changes} - [raices en {interval[2]}] {pos_changes} = {pos}')

    return pos, neg

In [7]:
pos, neg = numberOfRoots(P, interval = [-10, 0, 10])
(pos, neg)

Cambios de signo en -10 = [1], 0 = [0] y 10 = [2]
Raices positivas = [raices en -10] 1 - [raices en 0] 0 = 1
Raices negativas = [raices en 0] 0 - [raices en 10] 2 = -2


(-2, 1)

3. Utilizar la implementación del apartado 2 para **separar las raíces de un
polinomio** dado (es decir, encontrar intervalos que contengan una y solamente
una raíz). Hacer una función que devuelva una lista de intervalos.

NOTA: Asumir que todas las raíces del polinomio son reales y simples.


In [20]:
def SturmIntervals(P):
  intervals = list()
  test = [-10, 0, 10]
  pos, neg = numberOfRoots(P, test)
  iters = 0

  if (neg == 1):
    intervals.append([test[0], test[1]])

  if (pos == 1):
    intervals.append([test[1], test[2]])

  while (neg > 1) and (pos > 1):
    if (neg > 1):
      test = [test[0], (test[1] - test[0]) / 2, test[1]]
      pos, neg = numberOfRoots(P, test)
    
    if (pos > 1):
      test = [test[1], (test[2] - test[1]) / 2, test[2]]
      pos, neg = numberOfRoots(P, test)

    if (neg == 1):
      intervals.append([test[0], test[1]])

    if (pos == 1):
      intervals.append([test[1], test[2]])
    
    iters += 1
  return intervals


In [24]:
intervals = SturmIntervals(P)
intervals

Cambios de signo en -10 = [1], 0 = [0] y 10 = [2]
Raices negativas = [raices en -10] 1 - [raices en 0] 0 = 1
Raices positivas = [raices en 0] 0 - [raices en 10] 2 = -2


[[-10, 0]]

Calcular los intervalos de separación de las raíces de los siguientes polinomios:

$$P_1(x) = x^3-9x^2+24x-36$$
$$P_2(x) = x^6 - 9.5x^5 + 18x^4 + 33.5x^3 - 97x^2 + 18x +36$$

In [26]:
P_1 = np.array([1, -9, 24, -36])
P_2 = np.array([1, -9.5, 18, 33.5, -97, 18, 36])
i = SturmIntervals(P_1)
i2 = SturmIntervals(P_2)

print(i)
print(i2)

Cambios de signo en -10 = [1], 0 = [3] y 10 = [2]
Raices negativas = [raices en -10] 1 - [raices en 0] 3 = -2
Raices positivas = [raices en 0] 3 - [raices en 10] 2 = 1
Cambios de signo en -10 = [3], 0 = [3] y 10 = [3]
Raices negativas = [raices en -10] 3 - [raices en 0] 3 = 0
Raices positivas = [raices en 0] 3 - [raices en 10] 3 = 0
[[0, 10]]
[]
